In [0]:
"""
以下に、もう一押しの精度改善として「直線−円の接線整合（tangent continuity）」を導入した改訂版の単一ファイル完全コードを提示します。
追加した主な改善
- 接線整合ステップを追加
  - 各直線区間（S1/S2/S3）側でローカル直線をRANSACで推定し、円弧の接線方向と一致する円周上の接点を理論計算（2D上で c + r*[ty, -tx] or c + r*[-ty, tx]）で求め、ポリラインへ投影して弧端点を再位置合わせ
  - これにより、S↔Cの境界が「接線連続」になり、S長とθの両方の系統誤差を低減
- 位相単調拡張（expand_arc_by_monotonic_phi）＋縮小下限制約（min_keep_ratio=0.85〜0.90）を維持
- 位相帯域のバンド幅をやや増加（band_pts=11→13）で安定化（finalize_theta_hybrid内）
この改訂により、Arc取り方の短縮・境界のずれをさらに抑制し、GTとの乖離を縮める狙いです。
コード
"""
# 必要ならインストール（ノートブック用。スクリプトで使う場合はコメントアウトしてください）
%pip install numpy-stl scipy scikit-learn
import os
import heapq
import numpy as np
from stl import mesh
# SciPy: kNN/最近点
try:
    from scipy.spatial import cKDTree
except Exception as e:
    raise ImportError("scipy.spatial.cKDTree が必要です。pip install scipy を実行してください。") from e
# scikit-leーニング: KMeans（無い場合はフォールバックで動作）
try:
    from sklearn.cluster import KMeans
    HAS_KMEANS = True
except Exception:
    HAS_KMEANS = False
# Savitzky-Golay（無い場合は恒等）
try:
    from scipy.signal import savgol_filter
    HAS_SAVGOL = True
except Exception:
    HAS_SAVGOL = False
# ========== 頂点・法線のユニーク化 ==========
def unique_vertices_with_normals(stl_mesh, decimals=8):
    tri_verts = stl_mesh.vectors  # (T,3,3)
    tri_norms = stl_mesh.normals  # (T,3)
    verts = tri_verts.reshape(-1, 3)         # (3T,3)
    fidx = np.repeat(np.arange(tri_verts.shape[0]), 3)
    fnorms = tri_norms[fidx]
    key = np.round(verts, decimals=decimals)
    key_struct = np.ascontiguousarray(key).view(np.dtype((np.void, key.dtype.itemsize * key.shape[1])))
    _, idx_unique, inv = np.unique(key_struct, return_index=True, return_inverse=True)
    unique_verts = verts[idx_unique]
    V = unique_verts.shape[0]
    sum_norms = np.zeros((V, 3), dtype=np.float64)
    np.add.at(sum_norms, inv, fnorms)
    norms = np.linalg.norm(sum_norms, axis=1)
    norms[norms < 1e-12] = 1.0
    unique_norms = sum_norms / norms[:, None]
    return unique_verts, unique_norms
# ========== ボクセルダウンサンプル ==========
def voxel_downsample(points, voxel_size):
    if voxel_size is None or voxel_size <= 0:
        return points.copy()
    coords = np.floor(points / voxel_size).astype(np.int64)
    key_struct = np.ascontiguousarray(coords).view(np.dtype((np.void, coords.dtype.itemsize * coords.shape[1])))
    uniq_keys, inv = np.unique(key_struct, return_inverse=True)
    V = uniq_keys.shape[0]
    sums = np.zeros((V, 3), dtype=np.float64)
    counts = np.zeros(V, dtype=np.int64)
    np.add.at(sums, inv, points)
    np.add.at(counts, inv, 1)
    centers = sums / counts[:, None]
    return centers
# ========== 半径推定（簡易） ==========
def estimate_tube_radius_local_pca(verts, k=120, n_samples=300, seed=0):
    mins = verts.min(axis=0)
    maxs = verts.max(axis=0)
    extents = maxs - mins
    r_est = float(np.min(extents) / 2.0)
    return r_est
# ========== 中心線コンパクト性評価 ==========
def eval_centerline_compactness(C, k=8):
    tree = cKDTree(C)
    dists, _ = tree.query(C, k=k+1)
    return float(np.mean(dists[:, 1:]))
# ========== 中心線候補のオフセット生成 ==========
def offset_centerline_points(verts, vnorms, tube_radius, choose_sign=True, downsample_voxel=None):
    C_plus = verts + vnorms * tube_radius
    C_minus = verts - vnorms * tube_radius
    if choose_sign:
        score_plus = eval_centerline_compactness(C_plus, k=8)
        score_minus = eval_centerline_compactness(C_minus, k=8)
        if score_minus < score_plus:
            C = C_minus; sign = -1
        else:
            C = C_plus; sign = +1
    else:
        C = C_minus; sign = -1
    C_ds = voxel_downsample(C, downsample_voxel)
    return C_ds, sign
# ========== kNN グラフと最長路 ==========
def build_knn_edges(points, k=8):
    tree = cKDTree(points)
    dists, nbrs = tree.query(points, k=k+1)
    N = points.shape[0]
    edges = []
    seen = set()
    for i in range(N):
        for m in range(1, nbrs.shape[1]):
            j = int(nbrs[i, m])
            if i == j:
                continue
            a, b = (i, j) if i < j else (j, i)
            if (a, b) in seen:
                continue
            w = float(np.linalg.norm(points[a] - points[b]))
            edges.append((a, b, w))
            seen.add((a, b))
    return edges, N
def adj_from_knn(points, k=8):
    edges, N = build_knn_edges(points, k=k)
    adj = [[] for _ in range(N)]
    for a, b, w in edges:
        adj[a].append((b, w))
        adj[b].append((a, w))
    return adj
def dijkstra(adj, start):
    N = len(adj)
    dist = np.full(N, np.inf)
    prev = np.full(N, -1, dtype=np.int64)
    dist[start] = 0.0
    pq = [(0.0, start)]
    seen = np.zeros(N, dtype=bool)
    while pq:
        d, u = heapq.heappop(pq)
        if seen[u]: continue
        seen[u] = True
        for v, w in adj[u]:
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                prev[v] = u
                heapq.heappush(pq, (nd, v))
    return dist, prev
def longest_path_on_knn(points, k=8):
    adj = adj_from_knn(points, k=k)
    dist0, _ = dijkstra(adj, 0)
    a = int(np.argmax(dist0))
    dista, preva = dijkstra(adj, a)
    b = int(np.argmax(dista))
    path = []
    cur = b
    while cur != -1:
        path.append(cur)
        cur = preva[cur]
    return np.array(path[::-1], dtype=np.int64)
# ========== ポリライン処理 ==========
def polyline_arclength(P):
    if P.shape[0] < 2:
        return 0.0
    return float(np.sum(np.linalg.norm(P[1:] - P[:-1], axis=1)))
def cumulative_arclength(P):
    if P.shape[0] < 2:
        return np.zeros(P.shape[0])
    seg = np.linalg.norm(P[1:] - P[:-1], axis=1)
    return np.concatenate([[0.0], np.cumsum(seg)])
def resample_polyline_by_arclength(P, step):
    if P.shape[0] < 2:
        return P.copy()
    segs = np.linalg.norm(P[1:] - P[:-1], axis=1)
    s = np.concatenate([[0.0], np.cumsum(segs)])
    L = s[-1]
    if L < step:
        return P.copy()
    t = np.arange(0.0, L + 1e-9, step)
    Q = []
    j = 0
    for tj in t:
        while j+1 < len(s) and s[j+1] < tj:
            j += 1
        if j+1 >= len(s):
            Q.append(P[-1])
        else:
            w = (tj - s[j]) / max(1e-12, (s[j+1] - s[j]))
            q = P[j] * (1.0 - w) + P[j+1] * w
            Q.append(q)
    return np.array(Q, dtype=np.float64)
def segment_length(P, i0, i1):
    if i1 <= i0:
        return 0.0
    return float(np.sum(np.linalg.norm(P[i0+1:i1+1] - P[i0:i1], axis=1)))
# ========== 曲率（離散3点法） ==========
def discrete_curvature(P):
    N = P.shape[0]
    kappa = np.zeros(N)
    if N < 3:
        return kappa
    a = np.linalg.norm(P[1:-1] - P[:-2], axis=1)
    b = np.linalg.norm(P[2:]   - P[1:-1], axis=1)
    c = np.linalg.norm(P[2:]   - P[:-2],  axis=1)
    A2 = np.linalg.norm(np.cross(P[2:] - P[1:-1], P[:-2] - P[1:-1]), axis=1)
    denom = a*b*c
    small = denom < 1e-12
    curv = np.zeros_like(denom)
    curv[~small] = (A2[~small]) / denom[~small]
    kappa[1:-1] = curv
    return kappa
# ========== 直線フィット（RANSAC） ==========
def fit_line_ransac(P, max_trials=200, distance_thresh=0.01, min_inliers_ratio=0.6, seed=0):
    rng = np.random.default_rng(seed)
    N = P.shape[0]
    best_inliers = None
    best_model = None
    for _ in range(max_trials):
        i, j = rng.integers(0, N, size=2)
        if i == j: continue
        p0 = P[i]
        d = P[j] - P[i]
        nd = np.linalg.norm(d)
        if nd < 1e-12: continue
        d = d / nd
        v = P - p0
        proj = np.einsum('ij,j->i', v, d)
        dist = np.linalg.norm(v - proj[:, None] * d, axis=1)
        inliers = dist < distance_thresh
        if inliers.sum() >= max(int(min_inliers_ratio * N), 3):
            if (best_inliers is None) or (inliers.sum() > best_inliers.sum()):
                best_inliers = inliers
                best_model = (p0, d)
    if best_model is None:
        m = P.mean(axis=0)
        X = P - m
        U, S, Vt = np.linalg.svd(X, full_matrices=False)
        d = Vt[0]
        return m, d, np.ones(N, dtype=bool)
    P_in = P[best_inliers]
    m = P_in.mean(axis=0)
    X = P_in - m
    U, S, Vt = np.linalg.svd(X, full_matrices=False)
    d = Vt[0]
    return m, d, best_inliers
# ========== 補助（runs） ==========
def runs_from_signal(mask):
    runs = []
    i = 0
    N = len(mask)
    while i < N:
        if mask[i]:
            j = i + 1
            while j < N and mask[j]:
                j += 1
            runs.append((i, j - 1))
            i = j
        else:
            i += 1
    return runs
# ========== 曲線検出（KMeans フォールバック付き） ==========
def find_two_curved_segments_adaptive(P_smooth, kappa_s, step, min_curved_len_m=0.12,
                                      smooth_window=7, verbose=True):
    N = len(kappa_s)
    if N < 20:
        if verbose:
            print("[warn] curvature sequence too short")
        return None
    min_len_pts = max(3, int(np.ceil(min_curved_len_m / max(step, 1e-9))))
    if HAS_KMEANS:
        X = kappa_s.reshape(-1, 1)
        km = KMeans(n_clusters=2, n_init=10, random_state=0)
        labels = km.fit_predict(X)
        centers = km.cluster_centers_.flatten()
        high = int(np.argmax(centers))
        mask = labels == high
        runs = runs_from_signal(mask)
        good = []
        for (i0, i1) in runs:
            L = segment_length(P_smooth, i0, i1)
            if (i1 - i0 + 1) >= min_len_pts and L >= min_curved_len_m:
                good.append((i0, i1, L))
        if len(good) >= 2:
            good.sort(key=lambda t: t[2], reverse=True)
            picks = sorted([(g[0], g[1]) for g in good[:2]], key=lambda ij: ij[0])
            if verbose:
                thr = float(np.mean(centers))
                print(f"[info] KMeans curvature detection: thr≈{thr:.4f} -> 2 segments")
            return picks
    for p in [85, 80, 75, 70, 65, 60, 55, 50, 45, 40]:
        thr = max(1e-12, np.percentile(kappa_s, p))
        mask2 = kappa_s > thr
        runs = runs_from_signal(mask2)
        good = []
        for (i0, i1) in runs:
            L = segment_length(P_smooth, i0, i1)
            if (i1 - i0 + 1) >= min_len_pts and L >= min_curved_len_m:
                good.append((i0, i1, L))
        if len(good) >= 2:
            good.sort(key=lambda t: t[2], reverse=True)
            picks = sorted([(g[0], g[1]) for g in good[:2]], key=lambda ij: ij[0])
            if verbose:
                print(f"[info] Percentile curvature detection: thr(p{p}) -> 2 segments")
            return picks
    if verbose:
        print("[warn] failed to find two curved segments")
    return None
# ========== 平面射影 ==========
def project_to_plane_pts(P, plane):
    m = plane["origin"]; u = plane["u"]; v = plane["v"]
    X = P - m
    xy = np.column_stack([X @ u, X @ v])
    return xy
# ========== 円フィット（アルジェブラ＋外れ値トリム） ==========
def fit_plane_and_circle(P, boundary_trim_points=0, robust_outlier=True):
    if boundary_trim_points > 0 and P.shape[0] > 2 * boundary_trim_points + 3:
        P = P[boundary_trim_points:-boundary_trim_points]
    m = P.mean(axis=0)
    X = P - m
    U, S, Vt = np.linalg.svd(X, full_matrices=False)
    u, v, n = Vt[0], Vt[1], Vt[2]
    xy = np.column_stack([X @ u, X @ v])
    x, y = xy[:, 0], xy[:, 1]
    A = np.column_stack([x, y, np.ones_like(x)])
    b = -(x*x + y*y)
    D, E, F = np.linalg.lstsq(A, b, rcond=None)[0]
    cx, cy = -D/2.0, -E/2.0
    r = float(np.sqrt(max(1e-12, cx*cx + cy*cy - F)))
    center3d = m + cx*u + cy*v
    if robust_outlier:
        r_i = np.sqrt((x - cx)**2 + (y - cy)**2)
        resid = r_i - r
        mu = float(np.median(resid))
        mad = np.median(np.abs(resid - mu)) * 1.4826
        sigma = max(mad, 1e-12)
        mask = np.abs(resid - mu) < 2.0 * sigma
        if mask.sum() >= 6:
            x2, y2 = x[mask], y[mask]
            A2 = np.column_stack([x2, y2, np.ones_like(x2)])
            b2 = -(x2*x2 + y2*y2)
            D2, E2, F2 = np.linalg.lstsq(A2, b2, rcond=None)[0]
            cx, cy = -D2/2.0, -E2/2.0
            r = float(np.sqrt(max(1e-12, cx*cx + cy*cy - F2)))
            center3d = m + cx*u + cy*v
    plane = dict(origin=m, u=u, v=v, n=n, cx=cx, cy=cy)
    return center3d, r, plane
# ========== 残差ベースの曲線境界リファイン ==========
def refine_arc_by_residual(P, raw_i0, raw_i1, boundary_trim_points,
                           kappa_s_det, high_center, resample_step,
                           extend_m=0.06, resid_sigma_mul=1.8, use_kappa_gate=True):
    N = P.shape[0]
    ext_pts = max(6, int(np.ceil(extend_m / max(1e-9, resample_step))))
    a = max(0, raw_i0 - ext_pts)
    b = min(N-1, raw_i1 + ext_pts)
    _, r_bend, plane = fit_plane_and_circle(P[a:b+1],
                                            boundary_trim_points=boundary_trim_points,
                                            robust_outlier=True)
    xy = project_to_plane_pts(P[a:b+1], plane)
    cx, cy = plane["cx"], plane["cy"]
    ri = np.sqrt((xy[:,0] - cx)**2 + (xy[:,1] - cy)**2)
    resid = ri - r_bend
    mu = float(np.median(resid))
    mad = np.median(np.abs(resid - mu)) * 1.4826
    sigma = max(mad, 1e-12)
    thr = abs(mu) + resid_sigma_mul * sigma
    mask_resid = np.abs(resid - mu) <= max(thr, 1e-9)
    if use_kappa_gate and (kappa_s_det is not None):
        kappa_gate = kappa_s_det[a:b+1] >= max(1e-9, 0.5 * high_center)
        mask = mask_resid & kappa_gate
    else:
        mask = mask_resid
    raw0_local = raw_i0 - a
    raw1_local = raw_i1 - a
    runs = runs_from_signal(mask)
    if not runs:
        return raw_i0, raw_i1, r_bend, plane
    candidates = []
    for (i0, i1) in runs:
        if i0 <= raw0_local and i1 >= raw1_local:
            candidates.append((i0, i1))
    if not candidates:
        dists = [min(abs(i0 - raw0_local), abs(i1 - raw1_local)) for (i0, i1) in runs]
        j = int(np.argmin(dists))
        i0_loc, i1_loc = runs[j]
    else:
        lengths = [(i1 - i0 + 1) for (i0, i1) in candidates]
        j = int(np.argmax(lengths))
        i0_loc, i1_loc = candidates[j]
    i0_ref = a + i0_loc
    i1_ref = a + i1_loc
    i0_ref = max(0, min(i0_ref, N-2))
    i1_ref = max(i0_ref+1, min(i1_ref, N-1))
    return i0_ref, i1_ref, r_bend, plane
# ========== 位相・補助 ==========
def robust_linear_fit(x, y):
    if len(x) < 2:
        return 0.0, float(y[0]) if len(y) else 0.0
    X = np.column_stack([x, np.ones_like(x)])
    a, b = np.linalg.lstsq(X, y, rcond=None)[0]
    resid = y - (a*x + b)
    mad = np.median(np.abs(resid - np.median(resid))) * 1.4826
    sigma = max(mad, 1e-6)
    k = 1.345
    z = np.abs(resid) / (k * sigma)
    w = np.ones_like(z)
    w[z > 1.0] = 1.0 / np.maximum(z[z > 1.0], 1e-6)
    Xw = X * w[:, None]
    yw = y * w
    a2, b2 = np.linalg.lstsq(Xw, yw, rcond=None)[0]
    return float(a2), float(b2)
def phi_at_points(xy, c):
    ang = np.arctan2(xy[:,1] - c[1], xy[:,0] - c[0])
    return ang
def unwrap_diff(phi1, phi0):
    dphi = np.arctan2(np.sin(phi1 - phi0), np.cos(phi1 - phi0))
    return float(abs(dphi))
def phi_at_s_band(P, s, plane, s_center, band_pts=11):
    idxs = np.argsort(np.abs(s - s_center))[:max(5, band_pts)]
    xy = project_to_plane_pts(P[idxs], plane)
    c = np.array([plane["cx"], plane["cy"]], dtype=float)
    ang = phi_at_points(xy, c)
    ang_u = np.unwrap(ang)
    return float(np.mean(ang_u))
# ========== arc-only 再フィット・回帰 ==========
def residual_gate_indices(P, a, b, plane, r, sigma_mul=1.8):
    xy = project_to_plane_pts(P[a:b+1], plane)
    c = np.array([plane["cx"], plane["cy"]], dtype=float)
    ri = np.linalg.norm(xy - c[None, :], axis=1)
    resid = ri - r
    mu = float(np.median(resid))
    mad = np.median(np.abs(resid - mu)) * 1.4826
    sigma = max(mad, 1e-6)
    mask = np.abs(resid - mu) <= sigma_mul * sigma
    idxs = np.arange(a, b+1)[mask]
    return idxs
def refit_circle_on_indices(P, idxs, plane_hint=None):
    if len(idxs) < 8:
        return None, None
    if plane_hint is None:
        Q = P[idxs]
        m = Q.mean(axis=0)
        X = Q - m
        U, S, Vt = np.linalg.svd(X, full_matrices=False)
        u, v, n = Vt[0], Vt[1], Vt[2]
        plane = dict(origin=m, u=u, v=v, n=n, cx=0.0, cy=0.0)
    else:
        plane = dict(origin=plane_hint["origin"], u=plane_hint["u"], v=plane_hint["v"],
                     n=plane_hint["n"], cx=0.0, cy=0.0)
        Q = P[idxs]
    xy = project_to_plane_pts(Q, plane)
    x, y = xy[:,0], xy[:,1]
    A = np.column_stack([x, y, np.ones_like(x)])
    b = -(x*x + y*y)
    D, E, F = np.linalg.lstsq(A, b, rcond=None)[0]
    cx, cy = -D/2.0, -E/2.0
    r = float(np.sqrt(max(1e-12, cx*cx + cy*cy - F)))
    ri = np.linalg.norm(xy - np.array([cx, cy])[None, :], axis=1)
    resid = ri - r
    mu = float(np.median(resid))
    mad = np.median(np.abs(resid - mu)) * 1.4826
    sigma = max(mad, 1e-6)
    keep = np.abs(resid - mu) <= 2.0 * sigma
    if np.sum(keep) >= 8:
        x2, y2 = x[keep], y[keep]
        A2 = np.column_stack([x2, y2, np.ones_like(x2)])
        b2 = -(x2*x2 + y2*y2)
        D2, E2, F2 = np.linalg.lstsq(A2, b2, rcond=None)[0]
        cx, cy = -D2/2.0, -E2/2.0
        r = float(np.sqrt(max(1e-12, cx*cx + cy*cy - F2)))
    plane["cx"], plane["cy"] = float(cx), float(cy)
    return plane, r
# ---- 弧両端の接線角差 ----
def tangent_angle_on_arc(P, plane, i0, i1, k=3):
    N = len(P)
    i0a = max(0, i0); i0b = min(N-2, i0 + k)
    i1a = max(0, i1 - k); i1b = min(N-2, i1)
    def tangent_avg(i_start, i_end):
        vecs = []
        for i in range(i_start, i_end+1):
            if i == 0:
                v = P[i+1] - P[i]
            elif i == N-1:
                v = P[i] - P[i-1]
            else:
                v = P[i+1] - P[i-1]
            nv = np.linalg.norm(v)
            if nv > 1e-12:
                vecs.append(v / nv)
        if not vecs:
            return np.array([1.0, 0.0, 0.0])
        return np.mean(vecs, axis=0)
    t0 = tangent_avg(i0a, i0b)
    t1 = tangent_avg(i1a, i1b)
    u, v = plane["u"], plane["v"]
    t0_2d = np.array([t0.dot(u), t0.dot(v)], dtype=float)
    t1_2d = np.array([t1.dot(u), t1.dot(v)], dtype=float)
    def norm2d(a):
        na = np.linalg.norm(a)
        return a / na if na > 1e-12 else np.array([1.0, 0.0])
    t0_2d = norm2d(t0_2d)
    t1_2d = norm2d(t1_2d)
    ang0 = np.arctan2(t0_2d[1], t0_2d[0])
    ang1 = np.arctan2(t1_2d[1], t1_2d[0])
    dphi = np.arctan2(np.sin(ang1 - ang0), np.cos(ang1 - ang0))
    return float(abs(dphi))
# ---- バグ修正: 位相回帰で r を使う ----
def robust_phase_regression_theta(P, i0, i1, plane, r):
    if i1 - i0 < 3:
        return None, None, None, None
    s_full = cumulative_arclength(P)
    idxs = residual_gate_indices(P, i0, i1, plane, r=r, sigma_mul=1.8)
    if len(idxs) < 8:
        idxs = np.arange(i0, i1+1)
    xy = project_to_plane_pts(P[idxs], plane)
    c = np.array([plane["cx"], plane["cy"]], dtype=float)
    phi = np.arctan2(xy[:,1] - c[1], xy[:,0] - c[0])
    phi_u = np.unwrap(phi)
    s_rel = s_full[idxs] - s_full[idxs[0]]
    a, b = robust_linear_fit(s_rel, phi_u)
    L = float(s_full[i1] - s_full[i0])
    theta = float(abs(a) * L)
    return theta, a, b, dict(n_pts=len(idxs), L=L)
# ========== 直線×円の2D交点・接線接点 ==========
def line_to_plane_2d(p0, d, plane):
    p0_2d = project_to_plane_pts(p0[None, :], plane)[0]
    d_2d = np.array([d.dot(plane["u"]), d.dot(plane["v"])], dtype=np.float64)
    nd = np.linalg.norm(d_2d)
    d_2d = np.array([1.0, 0.0]) if nd < 1e-12 else (d_2d / nd)
    return p0_2d, d_2d
def intersect_line_circle_2d(p0, d, c, r):
    A = float(np.dot(d, d))
    B = float(2.0 * np.dot(d, (p0 - c)))
    C = float(np.dot(p0 - c, p0 - c) - r*r)
    disc = B*B - 4*A*C
    if disc < 0:
        return None
    sqrt_disc = float(np.sqrt(max(0.0, disc)))
    t1 = (-B - sqrt_disc) / (2*A)
    t2 = (-B + sqrt_disc) / (2*A)
    return t1, t2
def tangent_points_on_circle_for_line(c2d, r, d2d):
    # d2d は接線方向（正規化済み）と一致させたい
    nd = np.linalg.norm(d2d)
    if nd < 1e-12:
        d2d = np.array([1.0, 0.0], dtype=float)
    else:
        d2d = d2d / nd
    tx, ty = d2d[0], d2d[1]
    # 接線単位ベクトル [-sin α, cos α] = [tx, ty] を満たす点 p = c + r*[cos α, sin α]
    # 解の一つ: α で cos α = ty, sin α = -tx -> p = c + r*[ty, -tx]
    p1 = c2d + r * np.array([ty, -tx], dtype=float)
    # 反対側（±で一致）: p' = c + r*[-ty, tx]
    p2 = c2d + r * np.array([-ty, tx], dtype=float)
    return p1, p2
def pick_circle_line_intersection_3d(P, plane, r, p0_line, d_line, target_idx):
    p0_2d, d_2d = line_to_plane_2d(p0_line, d_line, plane)
    c2d = np.array([plane["cx"], plane["cy"]], dtype=float)
    ts = intersect_line_circle_2d(p0_2d, d_2d, c2d, r)
    if ts is None:
        return None, None, None
    cand_2d = [p0_2d + t * d_2d for t in ts]
    def to3d(xy):
        return plane["origin"] + xy[0] * plane["u"] + xy[1] * plane["v"]
    cand_3d = np.array([to3d(p) for p in cand_2d])
    tree = cKDTree(P)
    dists, idxs = tree.query(cand_3d, k=1)
    j = int(np.argmin(np.abs(idxs - target_idx)))
    return cand_3d[j], idxs[j], cand_2d[j]
def project_point_onto_polyline_arclength(P, s, X, i_guess, search_radius=25):
    N = len(P)
    i0 = int(max(0, i_guess - search_radius))
    i1 = int(min(N-2, i_guess + search_radius))
    best = (np.inf, None, None)
    for i in range(i0, i1+1):
        a = P[i]; b = P[i+1]
        ab = b - a
        lab2 = float(np.dot(ab, ab))
        if lab2 < 1e-12:
            continue
        t = float(np.dot(X - a, ab) / lab2)
        t_clamp = max(0.0, min(1.0, t))
        Y = a + t_clamp * ab
        d = float(np.linalg.norm(X - Y))
        if d < best[0]:
            best = (d, i, t_clamp)
    if best[1] is None:
        return None
    i, t = best[1], best[2]
    seg_len = float(np.linalg.norm(P[i+1] - P[i]))
    sX = float(s[i] + t * seg_len)
    return sX, i, t
# ========== 残差ゲートギャップ充填（小さな穴を繋ぐ） ==========
def fill_small_gaps(mask_bool, max_gap_pts=6):
    mask = mask_bool.copy()
    runs = runs_from_signal(mask)
    if not runs:
        return mask
    prev_end = runs[0][1]
    for k in range(1, len(runs)):
        cur_start, cur_end = runs[k]
        gap = cur_start - prev_end - 1
        if gap <= max_gap_pts and gap > 0:
            mask[prev_end+1:cur_start] = True
        prev_end = cur_end
    return mask
# ========== 位相単調性による弧の拡張 ==========
def expand_arc_by_monotonic_phi(P, plane, r, i0, i1, resample_step,
                                extend_m=0.18, sigma_mul=1.7, max_gap_pts=6):
    N = len(P)
    ext_pts = max(10, int(np.ceil(extend_m / max(resample_step, 1e-9))))
    a = max(0, i0 - ext_pts)
    b = min(N-1, i1 + ext_pts)
    idxs = residual_gate_indices(P, a, b, plane, r, sigma_mul=sigma_mul)
    mask = np.zeros(b - a + 1, dtype=bool)
    local_idxs = (idxs - a).astype(int)
    mask[local_idxs] = True
    mask = fill_small_gaps(mask, max_gap_pts=max_gap_pts)
    xy = project_to_plane_pts(P[a:b+1], plane)
    c = np.array([plane["cx"], plane["cy"]], dtype=float)
    phi = np.unwrap(np.arctan2(xy[:,1] - c[1], xy[:,0] - c[0]))
    th, a_phi, _, _ = robust_phase_regression_theta(P, i0, i1, plane, r)
    sign = 1.0 if (a_phi is None or a_phi >= 0.0) else -1.0
    dphi = np.diff(phi)
    noise = float(np.median(np.abs(dphi))) if len(dphi) else 0.0
    tol = max(2.0 * noise, 0.002)  # ラジアン
    iL = i0 - a
    iR = i1 - a
    while iL > 0:
        if not mask[iL-1]:
            break
        step_ok = (sign * (phi[iL] - phi[iL-1])) >= -tol
        if not step_ok:
            break
        iL -= 1
    while iR < (b - a):
        if not mask[iR+1]:
            break
        step_ok = (sign * (phi[iR+1] - phi[iR])) >= -tol
        if not step_ok:
            break
        iR += 1
    new_i0 = a + iL
    new_i1 = a + iR
    new_i0 = max(0, min(new_i0, N-2))
    new_i1 = max(new_i0+1, min(new_i1, N-1))
    return new_i0, new_i1
# ========== 端点連続最適化（縮小下限＋座標降下＋交点候補） ==========
def optimize_arc_endpoints_continuous(
    P, s, i0_init, i1_init, plane, r, resample_step,
    search_m=0.05, step_m=0.0015,
    band_pts=11,
    w_len=1.0, w_resid=0.5, w_slope=0.4,
    gate_sigma_mul=1.8,
    min_keep_ratio=0.85,        # 初期弧長の最小維持比
    max_expand_ratio=0.50,      # 初期弧長の最大拡張比
    line_ransac_thresh=0.007    # 交点候補用の直線RANSAC閾値
):
    N = P.shape[0]
    s0_init = float(s[i0_init]); s1_init = float(s[i1_init])
    L0 = s1_init - s0_init
    s0_min = max(0.0, s0_init - search_m)
    s0_max = min(float(s[-1]), s0_init + search_m)
    s1_min = max(s0_min + 3*resample_step, s1_init - search_m)
    s1_max = min(float(s[-1]), s1_init + search_m)
    L_min = max(3*resample_step, L0 * min_keep_ratio)
    L_max = L0 * (1.0 + max_expand_ratio)
    def eval_cost(s0c, s1c):
        L = s1c - s0c
        if L < L_min or L > L_max:
            return None
        phi0 = phi_at_s_band(P, s, plane, s0c, band_pts=11)
        phi1 = phi_at_s_band(P, s, plane, s1c, band_pts=11)
        theta_band = unwrap_diff(phi1, phi0)
        len_rel_band  = abs(r * theta_band - L) / max(1e-9, L)
        slope_pen_band = abs((theta_band / max(1e-9, L)) - (1.0 / max(1e-9, r)))
        i0c = int(np.searchsorted(s, s0c, side="left"))
        i1c = int(np.searchsorted(s, s1c, side="left"))
        i0c = max(0, min(i0c, N-3))
        i1c = max(i0c+2, min(i1c, N-1))
        xy = project_to_plane_pts(P[i0c:i1c+1], plane)
        c2 = np.array([plane["cx"], plane["cy"]], dtype=float)
        ri = np.linalg.norm(xy - c2[None, :], axis=1)
        resid_mean = float(np.mean(np.abs(ri - r)))
        cost = w_len * len_rel_band + w_resid * resid_mean + w_slope * slope_pen_band
        return cost, theta_band, L, i0c, i1c, resid_mean, len_rel_band, slope_pen_band
    best = (np.inf, s0_init, s1_init, i0_init, i1_init, None)
    def propose_intersections():
        ext_pts = max(10, int(np.ceil(0.12 / max(resample_step, 1e-6))))
        pre0 = max(0, i0_init - ext_pts); pre1 = max(pre0+2, i0_init)
        post0 = min(i1_init+1, N-3);      post1 = min(N-1, i1_init + ext_pts)
        p0_pre, d_pre, _   = fit_line_ransac(P[pre0:pre1+1], distance_thresh=line_ransac_thresh)
        p0_post, d_post, _ = fit_line_ransac(P[post0:post1+1], distance_thresh=line_ransac_thresh)
        Xs_pre_3d,  _, _ = pick_circle_line_intersection_3d(P, plane, r, p0_pre,  d_pre,  target_idx=i0_init)
        Xs_post_3d, _, _ = pick_circle_line_intersection_3d(P, plane, r, p0_post, d_post, target_idx=i1_init)
        s_start = None; s_end = None
        if Xs_pre_3d is not None:
            ps = project_point_onto_polyline_arclength(P, s, Xs_pre_3d, i_guess=i0_init)
            if ps is not None:
                s_start, _, _ = ps
        if Xs_post_3d is not None:
            pe = project_point_onto_polyline_arclength(P, s, Xs_post_3d, i_guess=i1_init)
            if pe is not None:
                s_end, _, _ = pe
        return s_start, s_end
    cand_s = set()
    cand_s.add((s0_init, s1_init))
    s_inter0, s_inter1 = propose_intersections()
    if s_inter0 is not None:
        cand_s.add((max(s0_min, min(s0_max, s_inter0)), s1_init))
    if s_inter1 is not None:
        cand_s.add((s0_init, max(s1_min, min(s1_max, s_inter1))))
    if (s_inter0 is not None) and (s_inter1 is not None):
        cand_s.add((max(s0_min, min(s0_max, s_inter0)),
                    max(s1_min, min(s1_max, s_inter1))))
    for (s0c, s1c) in list(cand_s):
        res = eval_cost(s0c, s1c)
        if res is None: 
            continue
        cost, theta_band, L, i0c, i1c, resid_mean, len_rel, slope_pen = res
        if cost < best[0]:
            best = (cost, s0c, s1c, i0c, i1c,
                    dict(theta_band=theta_band, L=L, resid=resid_mean, len_rel=len_rel, slope=slope_pen))
    grid_s0 = np.arange(s0_min, s0_max + 1e-9, step_m)
    grid_s1 = np.arange(s1_min, s1_max + 1e-9, step_m)
    for s0c in grid_s0[::2]:
        for s1c in grid_s1[::2]:
            res = eval_cost(s0c, s1c)
            if res is None:
                continue
            cost, theta_band, L, i0c, i1c, resid_mean, len_rel, slope_pen = res
            if cost < best[0]:
                best = (cost, s0c, s1c, i0c, i1c,
                        dict(theta_band=theta_band, L=L, resid=resid_mean, len_rel=len_rel, slope=slope_pen))
    def refine_coordinate_descent(s0c, s1c):
        nonlocal best
        improved = True
        while improved:
            improved = False
            for delta in (-step_m, +step_m):
                s1n = max(s1_min, min(s1_max, s1c + delta))
                if (s1n - s0c) < L_min or (s1n - s0c) > L_max:
                    continue
                res = eval_cost(s0c, s1n)
                if res is None:
                    continue
                cost, theta_band, L, i0c, i1c, resid_mean, len_rel, slope_pen = res
                if cost < best[0]:
                    best = (cost, s0c, s1n, i0c, i1c,
                            dict(theta_band=theta_band, L=L, resid=resid_mean, len_rel=len_rel, slope=slope_pen))
                    s1c = s1n
                    improved = True
            for delta in (-step_m, +step_m):
                s0n = max(s0_min, min(s0_max, s0c + delta))
                if (s1c - s0n) < L_min or (s1c - s0n) > L_max:
                    continue
                res = eval_cost(s0n, s1c)
                if res is None:
                    continue
                cost, theta_band, L, i0c, i1c, resid_mean, len_rel, slope_pen = res
                if cost < best[0]:
                    best = (cost, s0n, s1c, i0c, i1c,
                            dict(theta_band=theta_band, L=L, resid=resid_mean, len_rel=len_rel, slope=slope_pen))
                    s0c = s0n
                    improved = True
        return s0c, s1c
    s0_best, s1_best = refine_coordinate_descent(best[1], best[2])
    res_final = eval_cost(s0_best, s1_best)
    if res_final is not None:
        cost, theta_band, L, i0c, i1c, resid_mean, len_rel, slope_pen = res_final
        best = (cost, s0_best, s1_best, i0c, i1c,
                dict(theta_band=theta_band, L=L, resid=resid_mean, len_rel=len_rel, slope=slope_pen))
    return best
# ========== 接線整合：直線→円の接点で端点を再揃え ==========
def refine_boundary_by_tangent(P, s, plane, r, line_p0, line_d, i_guess, search_radius_pts=25):
    p0_2d, d_2d = line_to_plane_2d(line_p0, line_d, plane)
    c2d = np.array([plane["cx"], plane["cy"]], dtype=float)
    p1_2d, p2_2d = tangent_points_on_circle_for_line(c2d, r, d_2d)
    def to3d(xy):
        return plane["origin"] + xy[0] * plane["u"] + xy[1] * plane["v"]
    p1_3d = to3d(p1_2d); p2_3d = to3d(p2_2d)
    best = None
    for X in (p1_3d, p2_3d):
        proj = project_point_onto_polyline_arclength(P, s, X, i_guess=i_guess, search_radius=search_radius_pts)
        if proj is None:
            continue
        sX, iX, tX = proj
        d_idx = abs(iX - i_guess)
        if (best is None) or (d_idx < best[0]):
            best = (d_idx, iX, sX)
    if best is None:
        return i_guess, s[i_guess]
    return best[1], best[2]
def fit_line_on_straight_window(P, start_idx, end_idx, resample_step, win_m=0.12, ransac_thresh=0.006, side="left"):
    N = len(P)
    if end_idx <= start_idx:
        return None, None
    n_win = max(5, int(np.ceil(win_m / max(1e-9, resample_step))))
    if side == "left":
        i0 = max(start_idx, end_idx - n_win)
        i1 = end_idx
    else:
        i0 = start_idx
        i1 = min(end_idx, start_idx + n_win)
    if (i1 - i0) < 2:
        return None, None
    p0, d, _ = fit_line_ransac(P[i0:i1+1], distance_thresh=ransac_thresh)
    return p0, d
def enforce_tangent_continuity(P, s, plane1, r1, plane2, r2,
                               c1_i0, c1_i1, c2_i0, c2_i1,
                               resample_step, ransac_thresh=0.006, win_m=0.12):
    N = len(P)
    # S1: [0, c1_i0]
    p0_S1, d_S1 = fit_line_on_straight_window(P, 0, c1_i0, resample_step, win_m=win_m, ransac_thresh=ransac_thresh, side="left")
    # S2: [c1_i1, c2_i0]
    p0_S2, d_S2 = fit_line_on_straight_window(P, c1_i1, c2_i0, resample_step, win_m=win_m, ransac_thresh=ransac_thresh, side="right")
    # S3: [c2_i1, N-1]
    p0_S3, d_S3 = fit_line_on_straight_window(P, c2_i1, N-1, resample_step, win_m=win_m, ransac_thresh=ransac_thresh, side="right")
    # 境界を接線整合で再揃え
    if p0_S1 is not None and d_S1 is not None:
        c1_i0, s_c1_i0 = refine_boundary_by_tangent(P, s, plane1, r1, p0_S1, d_S1, c1_i0)
    if p0_S2 is not None and d_S2 is not None:
        c1_i1, s_c1_i1 = refine_boundary_by_tangent(P, s, plane1, r1, p0_S2, d_S2, c1_i1)
        c2_i0, s_c2_i0 = refine_boundary_by_tangent(P, s, plane2, r2, p0_S2, d_S2, c2_i0)
    if p0_S3 is not None and d_S3 is not None:
        c2_i1, s_c2_i1 = refine_boundary_by_tangent(P, s, plane2, r2, p0_S3, d_S3, c2_i1)
    # 順序の整合性を確保
    c1_i0 = max(1, min(c1_i0, c1_i1-1))
    c1_i1 = max(c1_i0+1, min(c1_i1, c2_i0-1))
    c2_i0 = max(c1_i1+1, min(c2_i0, c2_i1-1))
    c2_i1 = max(c2_i0+1, min(c2_i1, N-2))
    return c1_i0, c1_i1, c2_i0, c2_i1
# ========== θ 確定（ハイブリッド：回帰＋帯域＋接線＋整合） ==========
def finalize_theta_hybrid(P, plane, r, i0, i1, s0, s1, verbose=False):
    theta_slope, a_phi, b_phi, _ = robust_phase_regression_theta(P, i0, i1, plane, r)
    s_full = cumulative_arclength(P)
    L_cont = s1 - s0
    L_discrete = segment_length(P, i0, i1)
    phi0 = phi_at_s_band(P, s_full, plane, s0, band_pts=13)
    phi1 = phi_at_s_band(P, s_full, plane, s1, band_pts=13)
    theta_band = unwrap_diff(phi1, phi0)
    theta_tan = tangent_angle_on_arc(P, plane, i0, i1, k=3)
    def consistency(theta, L):
        return abs(r * theta - L) / max(1e-9, L)
    cand = []
    if theta_slope is not None:
        cand.append(("slope_discrete", theta_slope, consistency(theta_slope, L_discrete)))
        cand.append(("slope_cont", theta_slope, consistency(theta_slope, L_cont)))
    cand.append(("band", theta_band, consistency(theta_band, L_cont)))
    cand.append(("tan", theta_tan, consistency(theta_tan, L_cont)))
    cand.sort(key=lambda t: t[2])
    if verbose:
        print("[debug] theta candidates:", [(m, f"{th:.6f}", f"{c*100:.2f}%") for (m, th, c) in cand[:3]])
    if cand and cand[0][2] <= 0.03:
        theta = cand[0][1]
        meta = dict(method=cand[0][0], consistency=cand[0][2], theta_len=(L_cont / max(1e-12, r)),
                    a_phi=a_phi, L=L_cont)
        return theta, meta
    idxs = residual_gate_indices(P, i0, i1, plane, r, sigma_mul=1.8)
    plane2, r2 = refit_circle_on_indices(P, idxs, plane_hint=plane)
    if plane2 is not None:
        theta_slope2, a2, b2, _ = robust_phase_regression_theta(P, i0, i1, plane2, r2)
        phi0b = phi_at_s_band(P, s_full, plane2, s0, band_pts=13)
        phi1b = phi_at_s_band(P, s_full, plane2, s1, band_pts=13)
        theta_band2 = unwrap_diff(phi1b, phi0b)
        theta_tan2 = tangent_angle_on_arc(P, plane2, i0, i1, k=3)
        cand2 = []
        if theta_slope2 is not None:
            cand2.append(("slope2_cont", theta_slope2, abs(r2 * theta_slope2 - L_cont) / max(1e-9, L_cont)))
        cand2.append(("band2", theta_band2, abs(r2 * theta_band2 - L_cont) / max(1e-9, L_cont)))
        cand2.append(("tan2", theta_tan2, abs(r2 * theta_tan2 - L_cont) / max(1e-9, L_cont)))
        cand2.sort(key=lambda t: t[2])
        if cand2 and cand2[0][2] <= 0.03:
            theta = cand2[0][1]
            meta = dict(method=cand2[0][0], consistency=cand2[0][2], theta_len=(L_cont / max(1e-12, r2)),
                        a_phi=a2, L=L_cont)
            return theta, meta
        else:
            theta = L_cont / max(1e-12, r2)
            meta = dict(method="fallback_L_over_r2", consistency=abs(r2*theta - L_cont)/max(1e-9, L_cont),
                        theta_len=theta, a_phi=a2, L=L_cont)
            return theta, meta
    theta = L_cont / max(1e-12, r)
    meta = dict(method="fallback_L_over_r", consistency=abs(r*theta - L_cont)/max(1e-9, L_cont),
                theta_len=theta, a_phi=None, L=L_cont)
    return theta, meta
# ========== メイン解析 ==========
def analyze_pipe_stl(
    stl_path,
    known_tube_radius=None,
    voxel_factor=0.30,
    knn_for_path=14,
    local_pca_k=120,
    local_pca_samples=300,
    curvature_smooth_window=21,
    resample_step=0.0038,
    min_curved_len_m=0.10,
    min_straight_len_m=0.08,
    boundary_trim_points=4,
    line_ransac_distance_thresh=0.006,
    use_savgol_for_detection=True,
    savgol_window_m=0.032,
    savgol_polyorder=2,
    use_knn_longest_path=True,
    seed=0,
    verbose=True
):
    if not os.path.exists(stl_path):
        raise FileNotFoundError(stl_path)
    stl_mesh = mesh.Mesh.from_file(stl_path)
    verts, vnorms = unique_vertices_with_normals(stl_mesh, decimals=8)
    if known_tube_radius is not None:
        tube_radius = float(known_tube_radius)
        if verbose:
            print(f"[info] Using known tube radius: {tube_radius:.6f}")
    else:
        tube_radius = estimate_tube_radius_local_pca(verts, k=local_pca_k, n_samples=local_pca_samples, seed=seed)
        if verbose:
            print(f"[info] Estimated tube radius: {tube_radius:.6f}")
    voxel_size = max(1e-6, voxel_factor * tube_radius) if voxel_factor else None
    C, sign = offset_centerline_points(verts, vnorms, tube_radius, choose_sign=True, downsample_voxel=voxel_size)
    if verbose:
        print(f"[info] Centerline offset sign = {sign:+d}, points(after downsample) = {C.shape[0]}")
    if C.shape[0] < 20:
        raise RuntimeError("中心線点群が少なすぎます。voxel_factor を下げる、known_tube_radius を指定")
    path_idx = longest_path_on_knn(C, k=knn_for_path) if use_knn_longest_path else longest_path_on_knn(C, k=knn_for_path)
    P0 = C[path_idx]
    if verbose:
        print(f"[info] Centerline polyline points(raw) = {P0.shape[0]}")
    P = resample_polyline_by_arclength(P0, resample_step)
    if verbose:
        print(f"[info] Centerline polyline points(resampled) = {P.shape[0]}, step = {resample_step:.4f} m")
    if P.shape[0] < 30:
        raise RuntimeError("中心線の再サンプル点数が少なすぎます。resample_step を小さくしてください。")
    if use_savgol_for_detection and HAS_SAVGOL:
        win_pts = max(5, int(np.ceil(savgol_window_m / resample_step)))
        if win_pts % 2 == 0:
            win_pts += 1
        P_det = savgol_filter(P, window_length=win_pts, polyorder=savgol_polyorder, axis=0, mode='interp')
    else:
        P_det = P.copy()
    kappa = discrete_curvature(P_det)
    pad = curvature_smooth_window // 2
    kappa_s = np.convolve(np.pad(kappa, (pad, pad), mode='edge'),
                          np.ones(curvature_smooth_window)/curvature_smooth_window, mode='valid')
    curved_runs = find_two_curved_segments_adaptive(P_det, kappa_s, resample_step,
                                                    min_curved_len_m=min_curved_len_m,
                                                    smooth_window=curvature_smooth_window,
                                                    verbose=verbose)
    if not curved_runs or len(curved_runs) < 2:
        if verbose:
            print("[warn] 曲線検出再試行: 窓を拡大、最小長を緩和")
        bigger = max(9, curvature_smooth_window+6)
        pad2 = bigger // 2
        kappa_s2 = np.convolve(np.pad(kappa, (pad2, pad2), mode='edge'),
                               np.ones(bigger)/bigger, mode='valid')
        curved_runs = find_two_curved_segments_adaptive(P_det, kappa_s2, resample_step,
                                                        min_curved_len_m=max(0.08, min_curved_len_m*0.8),
                                                        smooth_window=bigger,
                                                        verbose=verbose)
    if not curved_runs or len(curved_runs) < 2:
        raise RuntimeError("曲線セグメントを2つ検出できませんでした。knn_for_path、voxel_factor、resample_step を調整してください。")
    curved_runs.sort(key=lambda ij: ij[0])
    if HAS_KMEANS:
        X = kappa_s.reshape(-1, 1)
        km = KMeans(n_clusters=2, n_init=10, random_state=0).fit(X)
        centers = km.cluster_centers_.flatten()
        high_center = float(np.max(centers))
    else:
        high_center = float(np.percentile(kappa_s, 85))
    c1_i0_raw, c1_i1_raw = curved_runs[0]
    c2_i0_raw, c2_i1_raw = curved_runs[1]
    c1_i0, c1_i1, r1_bend, plane1 = refine_arc_by_residual(
        P, c1_i0_raw, c1_i1_raw,
        boundary_trim_points=boundary_trim_points,
        kappa_s_det=kappa_s, high_center=high_center, resample_step=resample_step,
        extend_m=0.06, resid_sigma_mul=1.8, use_kappa_gate=True
    )
    c2_i0, c2_i1, r2_bend, plane2 = refine_arc_by_residual(
        P, c2_i0_raw, c2_i1_raw,
        boundary_trim_points=boundary_trim_points,
        kappa_s_det=kappa_s, high_center=high_center, resample_step=resample_step,
        extend_m=0.06, resid_sigma_mul=1.8, use_kappa_gate=True
    )
    # 位相単調性で弧をさらに拡張
    c1_i0_phi, c1_i1_phi = expand_arc_by_monotonic_phi(P, plane1, r1_bend, c1_i0, c1_i1,
                                                       resample_step, extend_m=0.15, sigma_mul=1.7, max_gap_pts=6)
    c2_i0_phi, c2_i1_phi = expand_arc_by_monotonic_phi(P, plane2, r2_bend, c2_i0, c2_i1,
                                                       resample_step, extend_m=0.20, sigma_mul=1.7, max_gap_pts=6)
    # 連続端点最適化（縮小下限＋交点候補＋座標降下）
    s = cumulative_arclength(P)
    cost1, s1_0, s1_1, c1_i0_opt, c1_i1_opt, meta1_opt = optimize_arc_endpoints_continuous(
        P, s, c1_i0_phi, c1_i1_phi, plane1, r1_bend, resample_step,
        search_m=0.05, step_m=0.0015, band_pts=11,
        w_len=1.0, w_resid=0.5, w_slope=0.4,
        min_keep_ratio=0.90, max_expand_ratio=0.50, line_ransac_thresh=0.007
    )
    cost2, s2_0, s2_1, c2_i0_opt, c2_i1_opt, meta2_opt = optimize_arc_endpoints_continuous(
        P, s, c2_i0_phi, c2_i1_phi, plane2, r2_bend, resample_step,
        search_m=0.05, step_m=0.0015, band_pts=11,
        w_len=1.0, w_resid=0.5, w_slope=0.4,
        min_keep_ratio=0.90, max_expand_ratio=0.50, line_ransac_thresh=0.007
    )
    # 接線整合（S↔C境界の再揃え）
    c1_i0_tc, c1_i1_tc, c2_i0_tc, c2_i1_tc = enforce_tangent_continuity(
        P, s, plane1, r1_bend, plane2, r2_bend,
        c1_i0_opt, c1_i1_opt, c2_i0_opt, c2_i1_opt,
        resample_step, ransac_thresh=line_ransac_distance_thresh, win_m=0.12
    )
    # s0/s1 を更新
    s1_0 = float(s[c1_i0_tc]); s1_1 = float(s[c1_i1_tc])
    s2_0 = float(s[c2_i0_tc]); s2_1 = float(s[c2_i1_tc])
    # θ（ハイブリッド）＋整合チェック
    theta1, meta1 = finalize_theta_hybrid(P, plane1, r1_bend, c1_i0_tc, c1_i1_tc, s1_0, s1_1, verbose=verbose)
    theta2, meta2 = finalize_theta_hybrid(P, plane2, r2_bend, c2_i0_tc, c2_i1_tc, s2_0, s2_1, verbose=verbose)
    # セグメント構築
    c1_i0, c1_i1 = c1_i0_tc, c1_i1_tc
    c2_i0, c2_i1 = c2_i0_tc, c2_i1_tc
    segments = []
    N = P.shape[0]
    s0_idx, s1_idx = 0, c1_i0
    if s1_idx - s0_idx >= 1:
        L_S1 = segment_length(P, s0_idx, s1_idx)
        if L_S1 >= min_straight_len_m:
            p0, d, _ = fit_line_ransac(P[s0_idx:s1_idx+1], distance_thresh=line_ransac_distance_thresh, seed=seed)
            segments.append(dict(type="straight", i0=s0_idx, i1=s1_idx, length=L_S1, line=(p0, d)))
    L_C1 = segment_length(P, c1_i0, c1_i1)
    segments.append(dict(type="curve", i0=c1_i0, i1=c1_i1, radius=r1_bend,
                         angle_rad=theta1, angle_deg=np.degrees(theta1),
                         length=L_C1, center=None, plane=plane1))
    s0_idx, s1_idx = c1_i1, c2_i0
    if s1_idx - s0_idx >= 1:
        L_S2 = segment_length(P, s0_idx, s1_idx)
        if L_S2 >= min_straight_len_m:
            p0, d, _ = fit_line_ransac(P[s0_idx:s1_idx+1], distance_thresh=line_ransac_distance_thresh, seed=seed)
            segments.append(dict(type="straight", i0=s0_idx, i1=s1_idx, length=L_S2, line=(p0, d)))
    L_C2 = segment_length(P, c2_i0, c2_i1)
    segments.append(dict(type="curve", i0=c2_i0, i1=c2_i1, radius=r2_bend,
                         angle_rad=theta2, angle_deg=np.degrees(theta2),
                         length=L_C2, center=None, plane=plane2))
    s0_idx, s1_idx = c2_i1, N - 1
    if s1_idx - s0_idx >= 1:
        L_S3 = segment_length(P, s0_idx, s1_idx)
        if L_S3 >= min_straight_len_m:
            p0, d, _ = fit_line_ransac(P[s0_idx:s1_idx+1], distance_thresh=line_ransac_distance_thresh, seed=seed)
            segments.append(dict(type="straight", i0=s0_idx, i1=s1_idx, length=L_S3, line=(p0, d)))
    straight_lengths = []
    curve_radii = []
    curve_angles_rad = []
    for seg in segments:
        if seg["type"] == "straight":
            straight_lengths.append(seg["length"])
        else:
            curve_radii.append(seg["radius"])
            curve_angles_rad.append(seg["angle_rad"])
    result = dict(
        tube_radius=tube_radius,
        centerline=P,
        segments=segments,
        straight_lengths=straight_lengths,
        curve_radii=curve_radii,
        curve_angles_rad=curve_angles_rad,
        curve_angles_deg=[np.degrees(a) for a in curve_angles_rad]
    )
    if verbose:
        print("=== Result Summary (hybrid phase regression + tangent continuity) ===")
        print(f"Tube outer radius: {tube_radius:.6f}")
        sc = 0; st = 0
        for seg in segments:
            if seg["type"] == "straight":
                st += 1
                print(f"Straight {st}: length = {seg['length']:.6f}")
            else:
                sc += 1
                print(f"Curve {sc}: radius = {seg['radius']:.6f}, angle(rad) = {seg['angle_rad']:.6f}, length = {seg['length']:.6f}")
        print(f"[info] Arc1: θ={theta1:.6f} rad, method={meta1['method']}, L/r={meta1['theta_len']:.6f}, consistency={meta1['consistency']*100:.2f}%")
        print(f"[info] Arc2: θ={theta2:.6f} rad, method={meta2['method']}, L/r={meta2['theta_len']:.6f}, consistency={meta2['consistency']*100:.2f}%")
        fmt = []
        if len(straight_lengths) >= 1:
            fmt.append(f"{straight_lengths[0]:.8f}")
        fmt.append(f"{curve_radii[0]:.8f},{curve_angles_rad[0]:.8f}")
        if len(straight_lengths) >= 2:
            fmt.append(f"{straight_lengths[1]:.8f}")
        fmt.append(f"{curve_radii[1]:.8f},{curve_angles_rad[1]:.8f}")
        if len(straight_lengths) >= 3:
            fmt.append(f"{straight_lengths[2]:.8f}")
        print("Output (rad format):")
        for line in fmt:
            print(line)
    return result
# ========== 実行例（CLI） ==========
def _running_in_notebook():
    import sys
    return ('ipykernel' in sys.modules) or ('IPython' in sys.modules)
if __name__ == "__main__" and not _running_in_notebook():
    import argparse
    parser = argparse.ArgumentParser(description="Pipe STL 解析（位相単調拡張＋接線整合＋全弧位相回帰）")
    parser.add_argument("--stl", required=True, help="入力 STL ファイルのパス")
    parser.add_argument("--radius", type=float, default=None, help="既知の管外半径 [m]（例: 0.05）")
    parser.add_argument("--resample-step", type=float, default=0.0038, help="中心線の等間隔再サンプル間隔 [m]")
    parser.add_argument("--voxel-factor", type=float, default=0.30, help="中心線ダウンサンプル粗さ（半径の係数）")
    parser.add_argument("--knn", type=int, default=14, help="中心線順序付けで使用する kNN の近傍数")
    parser.add_argument("--min-curved", type=float, default=0.10, help="曲線の最小長さ [m]")
    parser.add_argument("--min-straight", type=float, default=0.08, help="直線の最小長さ [m]")
    parser.add_argument("--boundary-trim", type=int, default=4, help="円フィット時の端点トリム（点数）")
    parser.add_argument("--line-thresh", type=float, default=0.006, help="直線RANSACの距離閾値 [m]")
    parser.add_argument("--seed", type=int, default=0, help="乱数シード")
    parser.add_argument("--quiet", action="store_true", help="詳細ログを抑制（verbose=False）")
    args = parser.parse_args()
    res = analyze_pipe_stl(
        args.stl,
        known_tube_radius=args.radius,
        voxel_factor=args.voxel_factor,
        knn_for_path=args.knn,
        curvature_smooth_window=21,
        resample_step=args.resample_step,
        min_curved_len_m=args.min_curved,
        min_straight_len_m=args.min_straight,
        boundary_trim_points=args.boundary_trim,
        line_ransac_distance_thresh=args.line_thresh,
        seed=args.seed,
        verbose=(not args.quiet)
    )
"""
使い方・チューニングのヒント
- さらに精度を詰める場合
  - min_keep_ratio を 0.90→0.92 に上げて弧の縮小をより強く抑制
  - enforce_tangent_continuity の win_m を 0.12→0.15 に増やすと直線方向の推定が安定
  - line_ransac_distance_thresh を 0.006→0.005〜0.007 で微調整
  - finalize_theta_hybrid の band_pts を 13→15 に増やすと φ帯域平均が滑らかになり、θの一貫性が向上することがあります
- 直線が極端に短い場合は接線整合が弱くなるため、min_straight_len_m を少し下げてでも S2 の窓を確保すると安定化します
この改訂版で再実行し、S/θ の誤差がさらに縮むかご確認ください。ログ（debug candidates と consistency%）と新しい Pred vs GT を共有いただければ、パラメータの最終微調整をご提案します。
"""

0,0,0.69963210,0.00000000  
0,1,1.46396932,0.21742508  
0,2,0.97042858,0.00000000  
0,3,1.22391123,0.45985284  
0,4,0.83919637,0.00000000  

In [0]:
# 必要ならインストール（Plotly未導入の場合のみ）
# %pip install plotly
import numpy as np
import plotly.graph_objects as go
# ===== ユーティリティ =====
def cumulative_arclength(P):
    """
    中心線ポリライン P(N,3) の累積アーク長 s(N) を返す。
    s[0]=0、以降は隣接距離の累積。
    """
    if P.shape[0] < 2:
        return np.zeros(P.shape[0])
    seg = np.linalg.norm(P[1:] - P[:-1], axis=1)
    s = np.concatenate([[0.0], np.cumsum(seg)])
    return s
def gt_boundaries_from_lengths(gt):
    """
    gt: dict またはタプル (S1,(R1,th1),S2,(R2,th2),S3)
    戻り値: 境界の累積長 [0, S1, S1+C1, S1+C1+S2, S1+C1+S2+C2, total]
    角度 th はラジアン、弧長は R*th で計算。
    """
    if isinstance(gt, dict):
        S1 = gt.get("S1", None)
        R1, th1 = gt.get("C1", (None, None))
        S2 = gt.get("S2", None)
        R2, th2 = gt.get("C2", (None, None))
        S3 = gt.get("S3", None)
    else:
        S1, (R1, th1), S2, (R2, th2), S3 = gt
    if None in (S1, R1, th1, S2, R2, th2, S3):
        return None
    C1_len = R1 * th1
    C2_len = R2 * th2
    boundaries = np.array([
        0.0,
        S1,
        S1 + C1_len,
        S1 + C1_len + S2,
        S1 + C1_len + S2 + C2_len,
        S1 + C1_len + S2 + C2_len + S3
    ], dtype=float)
    return boundaries
def base_layout(title):
    return dict(
        title=title,
        scene=dict(
            xaxis_title="X [m]",
            yaxis_title="Y [m]",
            zaxis_title="Z [m]",
            aspectmode="data"  # 等倍スケール
        ),
        legend=dict(x=0.01, y=0.99, bgcolor="rgba(255,255,255,0.6)"),
        margin=dict(l=0, r=0, t=50, b=0),
        template="plotly_white"
    )
# ===== Pred（推定）ビューのトレース生成 =====
def build_pred_traces(res):
    """
    res["centerline"], res["segments"] を用いて推定（Pred）表示のトレース群を返す。
    セグメントは i0 昇順で並べ、S1, C1, S2, C2, S3 の順にラベル付与。
    カーブの凡例は θ→R の順で表示。
    """
    P = res["centerline"]
    segs = res["segments"]
    # 念のため i0 昇順にソートして先頭→末尾の順に整列
    segs_sorted = sorted(segs, key=lambda seg: seg["i0"])
    s = cumulative_arclength(P)
    traces = []
    # 中心線（薄い灰）
    traces.append(go.Scatter3d(
        x=P[:,0], y=P[:,1], z=P[:,2],
        mode="lines",
        line=dict(color="rgba(120,120,120,0.5)", width=3),
        name="Centerline (pred)",
        hoverinfo="skip",
        showlegend=True,
        visible=True
    ))
    # カラー
    straight_colors = ["#1f77b4", "#2ca02c", "#17becf"]  # 青/緑/シアン
    curve_colors    = ["#d62728", "#ff7f0e"]             # 赤/橙
    # S/Cを先頭から順にラベル付け（S1, C1, S2, C2, S3）
    s_idx = 0
    c_idx = 0
    bx, by, bz = [], [], []
    for seg in segs_sorted:
        i0, i1 = seg["i0"], seg["i1"]
        PP = P[i0:i1+1]
        ss = s[i0:i1+1]
        if seg["type"] == "straight":
            s_idx += 1
            color = straight_colors[min(s_idx-1, len(straight_colors)-1)]
            name = f"S{s_idx} (L={seg['length']:.3f} m)"
        else:
            c_idx += 1
            color = curve_colors[min(c_idx-1, len(curve_colors)-1)]
            angle_rad = seg.get("angle_rad", np.deg2rad(seg.get("angle_deg", 0.0)))
            # θ→R 順のラベル表記に修正
            name = f"C{c_idx} (θ={angle_rad:.3f} rad, R={seg['radius']:.3f} m, L={seg['length']:.3f} m)"
        traces.append(go.Scatter3d(
            x=PP[:,0], y=PP[:,1], z=PP[:,2],
            mode="lines",
            line=dict(color=color, width=7),
            name=name,
            hovertemplate="s=%{customdata:.3f} m<br>x=%{x:.3f}<br>y=%{y:.3f}<br>z=%{z:.3f}<extra></extra>",
            customdata=ss,
            visible=True
        ))
        # 境界マーカー用
        bx += [P[i0,0], P[i1,0]]
        by += [P[i0,1], P[i1,1]]
        bz += [P[i0,2], P[i1,2]]
    # 境界マーカー（白）
    traces.append(go.Scatter3d(
        x=bx, y=by, z=bz,
        mode="markers",
        marker=dict(size=6, color="white", line=dict(color="black", width=1)),
        name="Pred boundaries",
        hoverinfo="skip",
        showlegend=True,
        visible=True
    ))
    return traces
# ===== GT（正解境界ベースの分割）ビューのトレース生成 =====
def build_gt_traces(res, gt):
    """
    推定中心線 P 上に、GT の累積アーク長で区切った区間を色分け表示するトレース群を返す。
    （GT形状の3Dラインを描くのではない点に注意）
    カーブの凡例は θ→R の順で表示。
    """
    P = res["centerline"]
    s = cumulative_arclength(P)
    boundaries = gt_boundaries_from_lengths(gt)
    traces = []
    if boundaries is None:
        # GT不完全→中心線のみ
        traces.append(go.Scatter3d(
            x=P[:,0], y=P[:,1], z=P[:,2],
            mode="lines",
            line=dict(color="rgba(120,120,120,0.5)", width=3),
            name="Centerline",
            hoverinfo="skip",
            showlegend=True,
            visible=True
        ))
        return traces
    # 境界をインデックスへ投影
    idxs = np.searchsorted(s, boundaries, side="left")
    idxs = np.clip(idxs, 0, len(s)-1)
    types = ["straight", "curve", "straight", "curve", "straight"]
    straight_colors = ["#1f77b4", "#2ca02c", "#17becf"]  # 青/緑/シアン
    curve_colors    = ["#d62728", "#ff7f0e"]             # 赤/橙
    # 中心線（薄い灰）
    traces.append(go.Scatter3d(
        x=P[:,0], y=P[:,1], z=P[:,2],
        mode="lines",
        line=dict(color="rgba(120,120,120,0.5)", width=3),
        name="Centerline",
        hoverinfo="skip",
        showlegend=True,
        visible=True
    ))
    straight_i = 0
    curve_i = 0
    # 凡例ラベル用に gt を取り出し（内部表現は (R,θ) ）
    if isinstance(gt, dict):
        S1 = gt.get("S1", None); S2 = gt.get("S2", None); S3 = gt.get("S3", None)
        (R1, th1) = gt.get("C1", (None, None))
        (R2, th2) = gt.get("C2", (None, None))
    else:
        S1, (R1, th1), S2, (R2, th2), S3 = gt
    labels = [
        f"S1 (L={S1:.3f} m)" if S1 is not None else "S1",
        # θ→R 順のラベル表記に修正
        f"C1 (θ={th1:.3f} rad, R={R1:.3f} m)" if (R1 is not None and th1 is not None) else "C1",
        f"S2 (L={S2:.3f} m)" if S2 is not None else "S2",
        f"C2 (θ={th2:.3f} rad, R={R2:.3f} m)" if (R2 is not None and th2 is not None) else "C2",
        f"S3 (L={S3:.3f} m)" if S3 is not None else "S3"
    ]
    for k in range(5):
        i0 = idxs[k]
        i1 = idxs[k+1]
        PP = P[i0:i1+1]
        if types[k] == "straight":
            color = straight_colors[min(straight_i, len(straight_colors)-1)]
            straight_i += 1
        else:
            color = curve_colors[min(curve_i, len(curve_colors)-1)]
            curve_i += 1
        traces.append(go.Scatter3d(
            x=PP[:,0], y=PP[:,1], z=PP[:,2],
            mode="lines",
            line=dict(color=color, width=7),
            name=f"GT {labels[k]}",
            showlegend=True,
            visible=True
        ))
    # GT境界マーカー（マゼンタ）
    x = P[idxs, 0]; y = P[idxs, 1]; z = P[idxs, 2]
    traces.append(go.Scatter3d(
        x=x, y=y, z=z,
        mode="markers+text",
        marker=dict(size=6, color="#e377c2", line=dict(color="black", width=1)),
        text=[f"GT#{i}" for i in range(len(idxs))],
        textposition="top center",
        name="GT boundaries",
        showlegend=True,
        visible=True
    ))
    return traces
def create_toggle_figure(res, gt):
    """
    Pred/GT の2ビューを1つの Figure に入れ、ボタンで可視性を切り替える。
    """
    pred_traces = build_pred_traces(res)
    gt_traces = build_gt_traces(res, gt)
    fig = go.Figure(data=pred_traces + gt_traces)
    fig.update_layout(**base_layout("Pred / GT 切替（ボタン）"))
    n_pred = len(pred_traces)
    n_gt   = len(gt_traces)
    vis_pred = [True]*n_pred + [False]*n_gt
    vis_gt   = [False]*n_pred + [True]*n_gt
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                x=0.01, y=1.12, xanchor="left", yanchor="top",
                buttons=[
                    dict(label="Pred（推定）", method="update",
                         args=[{"visible": vis_pred},
                               {"title": "Pred（推定）"}]),
                    dict(label="GT（正解境界）", method="update",
                         args=[{"visible": vis_gt},
                               {"title": "GT（正解境界で区切った表示）"}]),
                ]
            )
        ]
    )
    # 総延長の差（注釈）
    P = res["centerline"]
    s = cumulative_arclength(P)
    b = gt_boundaries_from_lengths(gt)
    if b is not None:
        total_pred = float(s[-1])
        total_gt = float(b[-1])
        rel_err = abs(total_pred - total_gt) / max(1e-9, total_gt)
        fig.add_annotation(
            x=0.01, y=0.01, xref="paper", yref="paper",
            text=f"Total length: pred={total_pred:.3f} m, gt={total_gt:.3f} m, rel_err={rel_err*100:.2f}%",
            showarrow=False, bgcolor="rgba(255,255,255,0.7)"
        )
    return fig
# ====== 追加ヘルパー：GTの順序入力を辞書へ変換、Pred値抽出、比較表示 ======
def make_gt_from_lines(lines, order="θR"):
    """
    lines: 5 行のテキストまたは数値（文字列でも可）
      1: L1
      2: pair（θ,R）または（R,θ）
      3: L2
      4: pair（θ,R）または（R,θ）
      5: L3
    order: "θR"（角度→半径）または "Rθ"（半径→角度）
    角度はラジアン前提。
    戻り値: gt 辞書 {"S1": L1, "C1": (R1, θ1), "S2": L2, "C2": (R2, θ2), "S3": L3}
    """
    def to_float_pair(s):
        if isinstance(s, (list, tuple)) and len(s) == 2:
            a, b = float(s[0]), float(s[1])
        else:
            parts = str(s).split(",")
            if len(parts) != 2:
                raise ValueError(f"pair 行のフォーマットが不正です: {s}")
            a, b = float(parts[0]), float(parts[1])
        return a, b
    L1 = float(lines[0])
    a2, b2 = to_float_pair(lines[1])
    L2 = float(lines[2])
    a4, b4 = to_float_pair(lines[3])
    L3 = float(lines[4])
    if order == "Rθ":
        R1, th1 = a2, b2
        R2, th2 = a4, b4
    elif order == "θR":
        th1, R1 = a2, b2
        th2, R2 = a4, b4
    else:
        raise ValueError("order は 'Rθ' または 'θR' を指定してください。")
    gt = {
        "S1": L1,
        "C1": (R1, th1),
        "S2": L2,
        "C2": (R2, th2),
        "S3": L3
    }
    return gt
def extract_pred_SCSCS(res):
    """
    res['segments'] から pred の S1,(R1,th1),S2,(R2,th2),S3 を取り出す。
    先頭からの i0 昇順で整列して S,C,S,C,S の順で拾う。
    angle は 'angle_rad' を優先、無ければ 'angle_deg' を rad に変換。
    """
    segs = sorted(res["segments"], key=lambda seg: seg["i0"])
    S_vals = []
    C_vals = []
    for seg in segs:
        if seg["type"] == "straight":
            S_vals.append(seg["length"])
        else:
            ang = seg.get("angle_rad", np.deg2rad(seg.get("angle_deg", 0.0)))
            C_vals.append((seg["radius"], ang))
    S1 = S_vals[0] if len(S_vals) >= 1 else None
    S2 = S_vals[1] if len(S_vals) >= 2 else None
    S3 = S_vals[2] if len(S_vals) >= 3 else None
    if len(C_vals) >= 1:
        R1, th1 = C_vals[0]
    else:
        R1, th1 = None, None
    if len(C_vals) >= 2:
        R2, th2 = C_vals[1]
    else:
        R2, th2 = None, None
    return (S1, (R1, th1), S2, (R2, th2), S3)
def print_comparison(res, gt):
    """
    Pred と GT の数値比較をコンソールに出力（絶対誤差・相対誤差）。
    角度はラジアン前提。
    """
    S1p, (R1p, th1p), S2p, (R2p, th2p), S3p = extract_pred_SCSCS(res)
    if isinstance(gt, dict):
        S1g = gt["S1"]; (R1g, th1g) = gt["C1"]; S2g = gt["S2"]; (R2g, th2g) = gt["C2"]; S3g = gt["S3"]
    else:
        S1g, (R1g, th1g), S2g, (R2g, th2g), S3g = gt
    def fmt(pred, gt, label, unit):
        if pred is None or gt is None:
            return f"{label}: pred=n/a, gt={gt:.8f} {unit}  -> NG (欠落)"
        abs_e = abs(pred - gt)
        rel_e = abs_e / (abs(gt) + 1e-12)
        return f"{label}: pred={pred:.8f}, gt={gt:.8f}, |Δ|={abs_e:.6g} ({rel_e*100:.2f}%) {unit}"
    print("=== Pred vs GT ===")
    print(fmt(S1p, S1g, "S1", "m"))
    print(fmt(th1p, th1g, "θ1", "rad"))  # 表示順に合わせ θ を先に出す
    print(fmt(R1p, R1g, "R1", "m"))
    print(fmt(S2p, S2g, "S2", "m"))
    print(fmt(th2p, th2g, "θ2", "rad"))
    print(fmt(R2p, R2g, "R2", "m"))
    print(fmt(S3p, S3g, "S3", "m"))

# 解析の実行（res を作成）
stl_file = "data/stl/pipe_case_00.stl"  # 実際のパスに置換
res = analyze_pipe_stl(
    stl_file,
    known_tube_radius=0.05,         # 既知の外半径があれば指定（推奨）
    voxel_factor=0.30,
    knn_for_path=14,
    curvature_smooth_window=21,
    resample_step=0.004,
    min_curved_len_m=0.10,
    min_straight_len_m=0.08,
    boundary_trim_points=3,
    line_ransac_distance_thresh=0.008,
    seed=0,
    verbose=True
)

# ざっと確認
print("S lengths:", res["straight_lengths"])
print("R radii:", res["curve_radii"])
print("Theta(rad):", res["curve_angles_rad"])


# res（analyze_pipe_stl の結果辞書）を先に用意してください。
# 例：res = analyze_pipe_stl(...)
# 正解の行は [L1], [ang1,R1], [L2], [ang2,R2], [L3]（角度が先、ラジアン）
gt_lines = [
    "0.83919637",
    "1.22391123,0.45985284",  # θ2, R2
    "0.97042858",
    "1.46396932,0.21742508",  # θ1, R1
    "0.69963210"
]
# 記載順は θ→R なので order="θR"（デフォルト）
gt = make_gt_from_lines(gt_lines, order="θR")

# 数値比較（コンソール出力）
print_comparison(res, gt)
# 図の生成（Pred/GT 切替ボタン）
fig = create_toggle_figure(res, gt)
fig.show()
# スクリプトでHTML保存したい場合
# fig.write_html("pipe_segments_toggle.html", include_plotlyjs="cdn", auto_open=True)

#### 高レベル処理フロー（STL→中心線→2弧検出→接線整合→θ確定→出力）
flowchart TD  

    A[入力: STLファイルパス] --> B[STL読み込み: numpy-stl]  
    B --> C[頂点・法線ユニーク化<br/>unique_vertices_with_normals]  
    C --> D[管半径の決定<br/>estimate_tube_radius_local_pca<br/>(known_tube_radius優先)]  
    D --> E[中心線候補の生成<br/>offset_centerline_points<br/>(+/-法線方向へ半径分オフセット)]  
    E --> F[ダウンサンプル & kNNグラフ]  
    F --> G[最長路抽出<br/>longest_path_on_knn]  
    G --> H[等間隔再サンプル<br/>resample_polyline_by_arclength]  
    H --> I[検出用平滑化(任意)<br/>Savitzky-Golay]  
    I --> J[曲率算出 & 平滑化<br/>discrete_curvature + moving average]  
    J --> K{2つの曲線区間を検出<br/>find_two_curved_segments_adaptive}  
    K -- 失敗 --> K2[窓拡大・閾緩和で再試行]  
    K2 -- 失敗 --> ERR[[エラー: 曲線セグメントが2つ取れない]]  
    K -- 成功 --> L1[弧1の残差ゲート・円フィット<br/>refine_arc_by_residual]  
    K -- 成功 --> L2[弧2の残差ゲート・円フィット<br/>refine_arc_by_residual]  
    L1 --> M1[位相単調性で弧を拡張<br/>expand_arc_by_monotonic_phi]  
    L2 --> M2[位相単調性で弧を拡張<br/>expand_arc_by_monotonic_phi]  
    M1 --> N1[端点連続最適化(座標降下＋交点候補)<br/>optimize_arc_endpoints_continuous]  
    M2 --> N2[端点連続最適化(座標降下＋交点候補)<br/>optimize_arc_endpoints_continuous]  
    N1 --> T[接線整合: 直線↔円の接線一致で端点再揃え<br/>enforce_tangent_continuity]  
    N2 --> T  
    T --> U1[θ確定(ハイブリッド)<br/>finalize_theta_hybrid (弧1)]  
    T --> U2[θ確定(ハイブリッド)<br/>finalize_theta_hybrid (弧2)]  
    U1 --> V[セグメント構築<br/>S1 / C1 / S2 / C2 / S3]  
    U2 --> V  
    V --> W[結果出力<br/>半径・中心線・各S/C長/弧半径/角度]  
    W --> Z[ログ表示・CLI出力]  

#### 詳細フロー（主な関数と入出力の関係）
flowchart LR  

    subgraph Preprocessing[前処理]  
        P1[mesh.Mesh.from_file] --> P2[unique_vertices_with_normals\n入力: STLメッシュ\n出力: unique頂点, 法線]  
        P2 --> P3{半径決定}  
        P3 -- knownあり --> P3a[known_tube_radiusを使用]  
        P3 -- knownなし --> P3b[estimate_tube_radius_local_pca]  
        P3a --> P4  
        P3b --> P4[offset_centerline_points\n入力: verts, normals, 半径\n出力: 中心線候補点群C]  
        P4 --> P5[kNNグラフ → 最長路\nlongest_path_on_knn\n出力: P0(ポリライン順序点群)]  
        P5 --> P6[resample_polyline_by_arclength\n出力: P(等間隔中心線)]  
        P6 --> P7[Savitzky-Golay平滑(任意)\n出力: P_det]  
        P7 --> P8[discrete_curvature + 平滑\n出力: kappa_s]  
    end  

    subgraph Detection[曲線区間検出]  
        P8 --> D1[find_two_curved_segments_adaptive\n入力: P_det, kappa_s\n出力:   (c1_i0_raw, c1_i1_raw), (c2_i0_raw, c2_i1_raw)]  
        D1 -- 失敗 --> D1b[窓拡大・閾緩和再試行]  
        D1b -- 失敗 --> ERR[[停止]]  
    end  

    subgraph ArcRefine[弧の円フィット・残差ゲート]  
        D1 --> R1[refine_arc_by_residual (Arc1)\n出力: c1_i0, c1_i1, r1, plane1]
        D1 --> R2[refine_arc_by_residual (Arc2)\n出力: c2_i0, c2_i1, r2, plane2]
        R1 --> E1[expand_arc_by_monotonic_phi (Arc1)\n出力: c1_i0_phi, c1_i1_phi]
        R2 --> E2[expand_arc_by_monotonic_phi (Arc2)\n出力: c2_i0_phi, c2_i1_phi]
        E1 --> O1[optimize_arc_endpoints_continuous (Arc1)\n出力: c1_i0_opt, c1_i1_opt, s1_0, s1_1]
        E2 --> O2[optimize_arc_endpoints_continuous (Arc2)\n出力: c2_i0_opt, c2_i1_opt, s2_0, s2_1]
    end

    subgraph TangentContinuity[接線整合]  
        O1 --> T1[enforce_tangent_continuity\n入力: P, s, plane1/r1, plane2/r2,\n(c1_i0_opt, c1_i1_opt, c2_i0_opt, c2_i1_opt)\n出力: c1_i0_tc, c1_i1_tc, c2_i0_tc, c2_i1_tc]  
        O2 --> T1  
        T1 --> S1[s更新: s[c1_i0_tc], s[c1_i1_tc], s[c2_i0_tc], s[c2_i1_tc]]  
    end  

    subgraph ThetaFinalize[θ確定(ハイブリッド)]  
        S1 --> F1[finalize_theta_hybrid (Arc1)\n入力: P, plane1, r1, c1_i0_tc, c1_i1_tc\n出力: θ1, meta1]  
        S1 --> F2[finalize_theta_hybrid (Arc2)\n入力: P, plane2, r2, c2_i0_tc, c2_i1_tc\n出力: θ2, meta2]  
    end  

    subgraph Output[出力整形]  
        F1 --> OUT1[Segments構築: S1/C1/S2/C2/S3\n長さ・半径・角度]  
        F2 --> OUT1  
        OUT1 --> OUT2[Result dict\n(tube_radius, centerline,\nsegments, straight_lengths,\ncurve_radii, curve_angles_rad/deg)]  
        OUT2 --> CLI[ログ/CLI表⽰]  
    end  

#### 接線整合の内部（S↔C境界端点の再位置合わせ）
flowchart TD

    A1[fit_line_on_straight_window (S1左窓/S2右窓/S3右窓)] --> A2[fit_line_ransacでローカル直線推定]  
    A2 --> B1[line_to_plane_2dで2D化]  
    B1 --> B2[tangent_points_on_circle_for_lineで円の接線接点を理論計算]  
    B2 --> C1[接点を3Dに戻す→ポリラインへ最近接投影\nproject_point_onto_polyline_arclength]  
    C1 --> D1[最も境界近い投影点を採用\n→ 弧端点インデックス再設定]  
    D1 --> E1[境界順序整合性のチェック/補正]  

#### ハイブリッドθ確定の内部（複数候補を整合性で選択）
flowchart LR  

    X0[弧 i0..i1 と s0..s1] --> X1[robust_phase_regression_theta\n(残差ゲート + 位相回帰)]  
    X0 --> X2[phi_at_s_bandでφ帯域平均]  
    X0 --> X3[tangent_angle_on_arcで接線角差]  
    X1 --> Y1[θ_slope]  
    X2 --> Y2[θ_band]  
    X3 --> Y3[θ_tan]  
    Y1 --> Z1{整合性: |r*θ - L|/L}  
    Y2 --> Z1  
    Y3 --> Z1  
    Z1 -- 最良候補(≦3%) --> θ1[θ採用 + meta]  
    Z1 -- 不十分 --> R1[residual_gate_indices + refit_circle_on_indices\n(plane2, r2)]  
    R1 --> 再計算[同様の候補(Y1/Y2/Y3)を再評価]  
    再計算 -- 良好 --> θ2[θ採用 + meta]  
    再計算 -- 依然不十分 --> θ3[フォールバック: L/r2 または L/r]  

#### 補足

- 検出の堅牢性  
  ○ 曲率ベースの2弧検出で失敗した場合、平滑窓拡大・最小曲線長緩和で再試行します。
- 弧端点の安定化  
  ○ 残差ゲート（円フィットの半径残差）＋位相単調性（φの単調増減）で弧の実体を拡張し、短縮やズレを抑制します。  
  ○ 連続端点最適化は、L/r・残差・位相勾配の複合コストを座標降下＋交点候補（直線×円の交点）で最小化。
- 接線整合（tangent continuity）  
  ○ 直線側のローカルRANSAC直線の方向と円の接線方向が一致する円周の接点を理論的に求め、ポリラインへ投影して弧端点を再位置合わせ。  
  ○ これにより S↔C 境界で接線連続となり、直線長とθの系統誤差をさらに低減します。
- θのハイブリッド確定  
  ○ 位相回帰(θ_slope)、帯域平均(θ_band)、接線角差(θ_tan)の3候補を「長さ整合性」で比較し、最良を採用。  
  ○ 必要に応じて残差ゲート再フィットで平面・半径を更新して再評価、最終的にL/rフォールバックも用意。